In [2]:
import numpy as np
import time

# Monte Carlo parameters
num_simulations = 100000  # Number of simulations
num_steps = 100  # Number of time steps
dt = 1.0 / num_steps  # Time step size

# Heston model parameters
S0 = 100  # Initial stock price
V0 = 0.04  # Initial variance
kappa = 2.0  # Mean reversion rate of variance
theta = 0.04  # Long-term variance
sigma = 0.3  # Volatility of volatility
rho = -0.7  # Correlation between the two Brownian motions
r = 0.03  # Risk-free rate
T = 1.0  # Time to maturity

def generate_correlated_brownian_motion(n_paths, n_steps, dt, rho):
    dW1 = np.random.randn(n_paths, n_steps) * np.sqrt(dt)
    dW2 = rho * dW1 + np.sqrt(1 - rho**2) * np.random.randn(n_paths, n_steps) * np.sqrt(dt)
    return dW1, dW2

def simulate_heston_model(n_paths, n_steps, dt, S0, V0, kappa, theta, sigma, rho, r, T):
    dW1, dW2 = generate_correlated_brownian_motion(n_paths, n_steps, dt, rho)
    
    S = np.zeros((n_paths, n_steps+1))
    V = np.zeros((n_paths, n_steps+1))
    
    S[:, 0] = S0
    V[:, 0] = V0
    
    for t in range(1, n_steps+1):
        V[:, t] = np.maximum(V[:, t-1] + kappa * (theta - V[:, t-1]) * dt + sigma * np.sqrt(V[:, t-1]) * dW2[:, t-1], 0)
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * V[:, t-1]) * dt + np.sqrt(V[:, t-1] * dt) * dW1[:, t-1])
    
    return S[:, -1]

def monte_carlo_pricing_heston(n_paths, n_steps, dt, S0, V0, kappa, theta, sigma, rho, r, T, K):
    np.random.seed(42)
    start_time = time.time()
    
    S_T = simulate_heston_model(n_paths, n_steps, dt, S0, V0, kappa, theta, sigma, rho, r, T)
    payoff = np.maximum(S_T - K, 0)
    
    price_estimate = np.exp(-r * T) * np.mean(payoff)
    price_std_error = np.exp(-r * T) * np.std(payoff) / np.sqrt(n_paths)
    relative_error = (price_std_error / price_estimate) * 100
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"Option Price: {price_estimate:.4f}")
    print(f"Standard Error: {price_std_error:.4f}")
    print(f"Relative Error: {relative_error:.2f}%")
    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    
    return price_estimate, price_std_error, relative_error, elapsed_time

# Running the Heston model MC simulation
K = 100  # Strike price
price, std_error, rel_error, elapsed_time = monte_carlo_pricing_heston(num_simulations, num_steps, dt, S0, V0, kappa, theta, sigma, rho, r, T, K)

# Print results
print(f"Final Option Price: {price:.4f}")
print(f"Final Standard Error: {std_error:.4f}")
print(f"Final Relative Error: {rel_error:.2f}%")
print(f"Total Elapsed Time: {elapsed_time:.2f} seconds")



Option Price: 1.6510
Standard Error: 0.0050
Relative Error: 0.30%
Elapsed Time: 0.94 seconds
Final Option Price: 1.6510
Final Standard Error: 0.0050
Final Relative Error: 0.30%
Total Elapsed Time: 0.94 seconds
